In [ ]:
# pip install bs4
# pip install requests

In [2]:
import requests # Requisição (acessar o site)
from bs4 import BeautifulSoup as bs # Tratar / Extrair dados html
import pandas as pd # Deixar a base estruturada
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
import time


## Entrando na página inicial do site com o Selenium 

In [2]:
url_site = 'https://www.auxiliadorapredial.com.br/'

In [3]:
driver = webdriver.Edge()

In [4]:
driver.get(url_site)

In [5]:
#Abrir lista de tipos de imóveis
tipo_imovel =driver.find_element('xpath','/html/body/div[1]/div[2]/div[2]/div[1]/div/div/div/form/div[1]')
time.sleep(2)
tipo_imovel.click()

In [6]:
#Clicar sobre o tipo de imóvel que queremos. Neste caso, 'Apartamento'
driver.find_element('xpath','//*[@id="menu-"]/div[3]/ul/li[4]').click()

In [7]:
#Localizando o campo de pesquisa da cidade e inserindo o nome da cidade
cidade = driver.find_element('xpath','/html/body/div[1]/div[2]/div[2]/div[1]/div/div/div/form/div[2]/div/div/div/input')


In [8]:
#Inserindo no campo de pesquisa a cidade
cidade.send_keys('Florianópolis - SC')

In [9]:
#Clicar no botão de busca
driver.find_element('xpath','/html/body/div[1]/div[2]/div[2]/div[1]/div/div/div/form/div[4]/button').click()

In [10]:
#Achando o elemento que contém os botões para navegar entre as páginas resultantes da pesquisa
number_pages = driver.find_element('xpath','/html/body/div[1]/div[2]/div/div/div[4]/nav/ul')

In [11]:
#Número total de páginas que serão extraídas as informações
last_page = number_pages.text.split('\n')[-1]

In [12]:
#Transformando a string obtida do elemento 'last_page' em tipo inteiro
last_page_int = int(last_page)


In [13]:
last_page_int

51

## Webscrapping function 

In [14]:
def info_apartments(apartamentos,base_imoveis):
    for index,apartamento in enumerate(apartamentos):
        
        try:
            reference = apartamento.find('div',{'class':'ref'}
            ).text.strip()
        except:
            reference = None
        
        try:
            preco = apartamento.find(
                            'span', {'class': 'fontSize16 bold green'}
            ).text.strip()
        except:
            preco = None
        #print(descricao.text.strip())

        try:
            endereco =  apartamento.find(
                            'div', {'class': 'RuaContainer'}
            ).text.strip()
        except:
            endereco = None
        #print(endereco.text.strip())

        try:
            bairro = apartamento.find(
                    'div', {'class': 'Location'}
            ).text.strip()
        except:
            bairro = None
        #print(preco.text.strip())
        #print('\n\n\n\n')
        metragem = None
        quartos = None
        garagens = None
        banheiros = None
        try:
            detalhes = apartamento.find(
                        'div',{'class': 'Details'}
                    )
            if detalhes is not None:
                tags = detalhes.find_all('img')
                values = detalhes.find_all('span')
                if tags and values:
                    for tag,value in zip(tags,values):
                                #print(tag['alt'],'\n',value.text)
                            if tag['alt'] == 'Metragem':
                                    metragem = value.text.strip()
                            elif tag['alt'] == 'Quartos':
                                    quartos = value.text.strip()
                            elif tag['alt'] == 'Garagens':
                                    garagens = value.text.strip()
                            elif tag['alt'] == 'Banheiros':
                                    banheiros = value.text.strip()
        except Exception as e:
            print(f"Erro ao extrair detalhes do apartamento: \n{e}")
     
     
        nova_linha = {'reference':reference,'preco': preco, 'endereco': endereco, 'localizacao': bairro,'metragem':metragem,'quartos':quartos,
                      'garagens':garagens,'banheiro':banheiros}
        ## Adicionando a nova linha ao DataFrame
        base_imoveis = base_imoveis.append(nova_linha, ignore_index=True)
    return base_imoveis

## Empty Dataframe

In [15]:
features = ['reference','preco','endereco','localizacao','metragem','quartos','garagens','banheiro']
base_imoveis = pd.DataFrame(columns=features)

In [16]:
base_imoveis

,reference,preco,endereco,localizacao,metragem,quartos,garagens,banheiro


## Executing the code 
Loop function that access each resulting page from the apartment's search at a specified city

In [17]:
user_agent = 'webscrappin/1.0'

In [18]:
cidade ='sc+florianopolis'

In [19]:
headers = {
                "User-Agent": user_agent
            } 


In [21]:
    for i in range(1,last_page_int+1):
        url = f'https://www.auxiliadorapredial.com.br/comprar/residencial/{cidade}?tipoImovel=Apartamento&page={i}'

    # Criar uma nova sessão para cada solicitação
        with requests.Session() as session:
            # Defina o cabeçalho User-Agent na solicitação
            headers = {
                "User-Agent": user_agent
            } 

            
            
            attempts = 0
            while attempts < 5:
                # Tentar fazer a solicitação
                try:
                    resposta = session.get(url, headers=headers)
                    resposta.raise_for_status()  # Lança uma exceção se a resposta for um erro HTTP
                    break
                    
                    
                except (requests.RequestException, ConnectionError) as e:
                    print(f'Erro na requisição de acesso a página {i}: {e}')
                    attempts +=1
                    #time.sleep(3)  # Aguarde 5 segundos antes de tentar novamente
                    driver.implicitly_wait(2)
                    #continue  # Pule para a próxima iteração do loop
                    
                    
                    
        if resposta.status_code == 200:
            #soup = bs(resposta.text)
            soup = bs(resposta.text, 'html.parser')
            apartamentos =soup.find_all('div', {'class': "sc-63846deb-0 iTvqzd"})
            soup_str = str(soup)
                    
            base_imoveis = info_apartments(apartamentos,base_imoveis)
            print(f'Página {i} processada.\n')
            
            
            
            
        else:
             print(f'Erro ao acessar a página {i}: \n{resposta.status_code}')
                
                
                
    #next_page = number_pages.find_element('xpath','//*[@id="contentSide"]/div[4]/nav/ul/li[9]/button').click()
    #print(url)

Erro na requisição de acesso a página 1: ('Connection aborted.', ConnectionResetError(10054, 'Foi forçado o cancelamento de uma conexão existente pelo host remoto', None, 10054, None))
Página 1 processada.

Erro na requisição de acesso a página 2: ('Connection aborted.', ConnectionResetError(10054, 'Foi forçado o cancelamento de uma conexão existente pelo host remoto', None, 10054, None))
Página 2 processada.

Página 3 processada.

Erro na requisição de acesso a página 4: ('Connection aborted.', ConnectionResetError(10054, 'Foi forçado o cancelamento de uma conexão existente pelo host remoto', None, 10054, None))
Erro na requisição de acesso a página 4: ('Connection aborted.', ConnectionResetError(10054, 'Foi forçado o cancelamento de uma conexão existente pelo host remoto', None, 10054, None))
Erro na requisição de acesso a página 4: ('Connection aborted.', ConnectionResetError(10054, 'Foi forçado o cancelamento de uma conexão existente pelo host remoto', None, 10054, None))
Erro na r

In [22]:
base_imoveis

,reference,preco,endereco,localizacao,metragem,quartos,garagens,banheiro
0,ref: 472082,R$ 900.000,rua dos golfinhos,"Ingleses do Rio Vermelho, Florianópolis - SC",61m²,2,1,2
1,ref: 468398,R$ 850.000,rodovia joão paulo,"João Paulo, Florianópolis - SC",68m²,2,1,2
2,ref: 468996,R$ 1.230.000,rua professor ayrton roberto de oliveira,"Itacorubi, Florianópolis - SC",89m²,3,1,2
3,ref: 465833,R$ 1.338.633,rua das dunas,"Ingleses do Rio Vermelho, Florianópolis - SC",117m²,3,2,3
4,ref: 465834,R$ 1.401.049,rua das dunas,"Ingleses do Rio Vermelho, Florianópolis - SC",125m²,3,2,4
...,...,...,...,...,...,...,...,...
1524,ref: 446500,R$ 790.000,rua capitão romualdo de barros,"Saco dos limões, Florianópolis - SC",63m²,2,1,3
1525,ref: 446510,R$ 2.300.000,avenida professor othon gama d'eça,"Centro, Florianópolis - SC",264m²,4,2,6
1526,ref: 446511,R$ 1.270.000,rodovia tertuliano brito xavier,"Jurere Leste, Florianópolis - SC",186m²,3,2,6
1527,ref: 446534,R$ 730.000,rodovia joão paulo,"João Paulo, Florianópolis - SC",76m²,2,1,3


# Standardizing the strings


In [53]:
base_imoveis = pd.read_excel('V3_base_imoveis_Sao_Paulo.xlsx').drop(columns = 'Unnamed: 0')

In [54]:
base_imoveis

,Unnamed: 0.1,reference,preco,endereco,localizacao,metragem,quartos,garagens,banheiro
0,0.0,ref: 1160319,R$ 225.000,rua das savelhas,"Conjunto Residencial Sitio Oratório, São Paulo...",65m²,2.0,1.0,1
1,1.0,ref: 1161935,R$ 210.000,rua das savelhas,"Conjunto Residencial Sitio Oratório, São Paulo...",55m²,2.0,NaN,1
2,2.0,ref: 1077734,R$ 695.000,rua campos vergueiro,"Vila Anastácio, São Paulo - SP",59m²,1.0,1.0,2
3,3.0,ref: 1050264,R$ 639.000,rua salvador pires de lima,"Sacomã, São Paulo - SP",64m²,2.0,1.0,2
4,4.0,ref: 25359,R$ 570.000,rua jose de oliveira coelho,"Vila Andrade, São Paulo - SP",69m²,2.0,2.0,2
...,...,...,...,...,...,...,...,...,...
48325,NaN,ref: 1023914,R$ 320.000,rua afonso pena,"Bom Retiro, São Paulo - SP",43m²,2.0,NaN,1
48326,NaN,ref: 1024188,R$ 400.000,rua vinte e cinco de janeiro,"Bom Retiro, São Paulo - SP",47m²,2.0,1.0,1
48327,NaN,ref: 1024191,R$ 376.363,rua newton prado,"Bom Retiro, São Paulo - SP",90m²,2.0,NaN,1
48328,NaN,ref: 1024201,R$ 370.000,avenida tiradentes,"Bom Retiro, São Paulo - SP",58m²,2.0,1.0,1


In [35]:
import unicodedata

def remover_acentos(texto):
    texto_sem_acentos = ''.join(
        (c for c in unicodedata.normalize('NFD', texto) if unicodedata.category(c) != 'Mn')
    )
    return texto_sem_acentos

Ola, como esta voce?


In [55]:
base_imoveis_aux = base_imoveis

In [56]:
base_imoveis_aux['localizacao'][0]

'Conjunto Residencial Sitio Oratório, São Paulo - SP'

In [57]:
for index,i in enumerate(base_imoveis_aux['endereco']):
    sem_acento = remover_acentos(i)
    base_imoveis_aux['endereco'][index] = sem_acento

C:\Users\loren\AppData\Local\Temp\ipykernel_2092\1833175048.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  base_imoveis_aux['endereco'][index] = sem_acento


In [58]:
for index,i in enumerate(base_imoveis_aux['localizacao']):
    sem_acento = remover_acentos(i)
    base_imoveis_aux['localizacao'][index] = sem_acento

C:\Users\loren\AppData\Local\Temp\ipykernel_2092\7421034.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  base_imoveis_aux['localizacao'][index] = sem_acento


In [59]:
base_imoveis_aux['localizacao'][0]

'Conjunto Residencial Sitio Oratorio, Sao Paulo - SP'

# Removing duplicates

In [60]:
base_imoveis_aux.shape

(48330, 9)

In [61]:
base_imoveis.duplicated().sum()

2370

In [62]:

base_imoveis_aux = base_imoveis_aux.drop_duplicates()

In [63]:
base_imoveis_aux.shape

(45960, 9)

# Salvando em formato CSV e excel

In [64]:
base_imoveis_aux.to_excel('V3_base_imoveis_Sao_Paulo_sem_acento.xlsx',index = True)

In [65]:
base_imoveis_aux.to_csv('V3_base_imoveis_Sao_Paulo_sem_acento.csv',index = True)

# Criando função que captura o IPTU e condomínio de cada apartamento

In [ ]:
# referencia


In [ ]:
def other_infos(base_imoveis):
        
        try:
            reference = soup.find('div',{'class':'ref'}
            ).text.strip()
        except:
            reference = None
        
        try:
            condominio = soup.find(
                            'span', {'class': 'fontSize16 bold green'}
            ).text.strip()
        except:
            condominio = None
        #print(descricao.text.strip())

        try:
            iptu =  soup.find(
                            'div', {'class': 'RuaContainer'}
            ).text.strip()
        except:
            iptu = None
     
        nova_linha = {'reference':reference,'condominio':condominio,'IPTU':iptu}
        ## Adicionando a nova linha ao DataFrame
        base_imoveis = base_imoveis.append(nova_linha, ignore_index=True)
    return base_imoveis

In [276]:
resposta = requests.get('https://www.auxiliadorapredial.com.br/imovel/venda/258334',headers=headers)

In [124]:
soup = bs(resposta.text, 'html.parser')

In [277]:
# Encontrar todos os elementos <div> da camada 1 com classe específica
camada_1 = soup.select('div', class_='MuiGrid-root MuiGrid-container MuiGrid-spacing-xs-10 css-12ikp2y')
elementos_not_find_1 = []

# Iterar sobre os elementos encontrados na camada 1
for idx_camada_1, element_camada_1 in enumerate(camada_1):
    # Encontrar os elementos <div> da camada 2 dentro de cada elemento da camada 1
    camada_2 = element_camada_1.select('div', class_='MuiGrid-root MuiGrid-item MuiGrid-grid-md-6 css-shiley')
    
    # Iterar sobre os elementos encontrados na camada 2 dentro deste elemento da camada 1
    for idx_camada_2, element_camada_2 in enumerate(camada_2):
        # Encontrar os elementos <div> da camada 3 dentro de cada elemento da camada 2
        camada_3 = element_camada_2.select('div', class_='MuiBox-root css-3vcsli')
        
        # Iterar sobre os elementos encontrados na camada 3 dentro deste elemento da camada 2
        for idx_camada_3, element_camada_3 in enumerate(camada_3):
            # Encontrar os elementos <div> da camada 4 dentro de cada elemento da camada 3
            camada_4 = element_camada_3.select('div', class_='MuiBox-root css-1bajd3f')
            
            # Verificar se há elementos na camada 4
            for idx_camada_4, element_camada_4 in enumerate(camada_4):
                camada_5 = element_camada_4.find_all('span')
                
                if camada_5:
                    # Iterar sobre os elementos encontrados na camada 4 dentro deste elemento da camada 3
                    for idx_camada_5, element_camada_5 in enumerate(camada_5):
                        # Imprimir o elemento encontrado na camada 4 juntamente com seu índice
                        
                        print(f"\nElemento encontrado na camada 5 (Caminho:  {idx_camada_1+1} ->{idx_camada_2+1} ->{idx_camada_3+1} ->{idx_camada_4+1}->{idx_camada_5+1}:")
                        print(element_camada_5)
                else:
                    #print("Nenhum elemento encontrado na camada 5 para este elemento da camada 4.")
                    pass
        # Verificar se há elementos na camada 3
        if not camada_3:
            #print("Nenhum elemento encontrado na camada 3 para este elemento da camada 2.")
            pass
    # Verificar se há elementos na camada 2
    if not camada_2:
        #print("Nenhum elemento encontrado na camada 2 para este elemento da camada 1.")
        pass


Elemento encontrado na camada 5 (Caminho:  1 ->2 ->1 ->57->1:
<span class="MuiTypography-root MuiTypography-caption css-128iows">© Auxiliadora Predial Ltda. Todos direitos reservados. CRECI RS - J43 | CRECI SP - J21663 | Política de privacidade</span>


## Executando o código

In [ ]:
    for i in list_reference:
        url = f'https://www.auxiliadorapredial.com.br/imovel/venda/{i}'

    # Criar uma nova sessão para cada solicitação
        with requests.Session() as session:
            # Defina o cabeçalho User-Agent na solicitação
            headers = {
                "User-Agent": user_agent
            } 

            
            
            attempts = 0
            while attempts < 5:
                # Tentar fazer a solicitação
                try:
                    resposta = session.get(url, headers=headers)
                    resposta.raise_for_status()  # Lança uma exceção se a resposta for um erro HTTP
                    break
                    
                    
                except (requests.RequestException, ConnectionError) as e:
                    print(f'Erro na requisição de acesso a página {i}: {e}')
                    attempts +=1
                    #time.sleep(3)  # Aguarde 5 segundos antes de tentar novamente
                    driver.implicitly_wait(2)
                    #continue  # Pule para a próxima iteração do loop
                    
                    
                    
        if resposta.status_code == 200:
            #soup = bs(resposta.text)
            soup = bs(resposta.text, 'html.parser')
                    
            base_imoveis = other_infos(base_imoveis)
            print(f'Página {i} processada.\n')
            
            
            
            
        else:
             print(f'Erro ao acessar a página {i}: \n{resposta.status_code}')
                
                
                
    #next_page = number_pages.find_element('xpath','//*[@id="contentSide"]/div[4]/nav/ul/li[9]/button').click()
    #print(url)